In [1]:
import math
import os
import random

import git
import imageio
import magnum as mn
import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt

# function to display the topdown map
from PIL import Image

import habitat_sim
from habitat_sim.utils import common as utils
from habitat_sim.utils import viz_utils as vut

repo = git.Repo(".", search_parent_directories=True)
dir_path = repo.working_tree_dir
data_path = os.path.join(dir_path, "data")  # Please check data folder, before you intialize the code
print(f"data_path = {data_path}")

data_path = /home/sohithvishnu/Documents/AI_Project/habitat-sim/data


/home/sohithvishnu/anaconda3/envs/habitat/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# @title Define Observation Display Utility Function { display-mode: "form" }

# @markdown A convenient function that displays sensor observations with matplotlib.

# @markdown (double click to see the code)


# Change to do something like this maybe: https://stackoverflow.com/a/41432704
def display_sample(rgb_obs, semantic_obs=np.array([]), depth_obs=np.array([])):
    from habitat_sim.utils.common import d3_40_colors_rgb

    rgb_img = Image.fromarray(rgb_obs, mode="RGBA")

    arr = [rgb_img]
    titles = ["rgb"]
    if semantic_obs.size != 0:
        semantic_img = Image.new("P", (semantic_obs.shape[1], semantic_obs.shape[0]))
        semantic_img.putpalette(d3_40_colors_rgb.flatten())
        semantic_img.putdata((semantic_obs.flatten() % 40).astype(np.uint8))
        semantic_img = semantic_img.convert("RGBA")
        arr.append(semantic_img)
        titles.append("semantic")

    if depth_obs.size != 0:
        depth_img = Image.fromarray((depth_obs / 10 * 255).astype(np.uint8), mode="L")
        arr.append(depth_img)
        titles.append("depth")

    plt.figure(figsize=(12, 8))
    for i, data in enumerate(arr):
        ax = plt.subplot(1, 3, i + 1)
        ax.axis("off")
        ax.set_title(titles[i])
        plt.imshow(data)
    plt.show(block=False)


if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument("--no-display", dest="display", action="store_false")
    parser.add_argument("--no-make-video", dest="make_video", action="store_false")
    parser.set_defaults(show_video=True, make_video=True)
    args, _ = parser.parse_known_args()
    show_video = args.display
    display = args.display
    do_make_video = args.make_video
else:
    show_video = False
    do_make_video = False
    display = False

# import the maps module alone for topdown mapping
if display:
    from habitat.utils.visualizations import maps

PluginManager::Manager: duplicate static plugin StbImageImporter, ignoring
PluginManager::Manager: duplicate static plugin GltfImporter, ignoring
PluginManager::Manager: duplicate static plugin BasisImporter, ignoring
PluginManager::Manager: duplicate static plugin AssimpImporter, ignoring
PluginManager::Manager: duplicate static plugin AnySceneImporter, ignoring
PluginManager::Manager: duplicate static plugin AnyImageImporter, ignoring


In [3]:
# This is the scene we are going to load.
# we support a variety of mesh formats, such as .glb, .gltf, .obj, .ply
test_scene = os.path.join(
    data_path, "scene_datasets/mp3d/17DRP5sb8fy/17DRP5sb8fy.glb" # This is data input part, please change it. 
)                                                                # You can download from the data link in the readme. 

sim_settings = {
    "scene": test_scene,  # Scene path
    "default_agent": 0,  # Index of the default agent
    "sensor_height": 1.5,  # Height of sensors in meters, relative to the agent
    "width": 256,  # Spatial resolution of the observations
    "height": 256,
}

In [4]:
# This function generates a config for the simulator.
# It contains two parts:
# one for the simulator backend
# one for the agent, where you can attach a bunch of sensors
def make_simple_cfg(settings):
    # simulator backend
    sim_cfg = habitat_sim.SimulatorConfiguration()
    sim_cfg.scene_id = settings["scene"]

    # agent
    agent_cfg = habitat_sim.agent.AgentConfiguration()

    # In the 1st example, we attach only one sensor,
    # a RGB visual sensor, to the agent
    rgb_sensor_spec = habitat_sim.CameraSensorSpec()
    rgb_sensor_spec.uuid = "color_sensor"
    rgb_sensor_spec.sensor_type = habitat_sim.SensorType.COLOR
    rgb_sensor_spec.resolution = [settings["height"], settings["width"]]
    rgb_sensor_spec.position = [0.0, settings["sensor_height"], 0.0]

    agent_cfg.sensor_specifications = [rgb_sensor_spec]

    return habitat_sim.Configuration(sim_cfg, [agent_cfg])


cfg = make_simple_cfg(sim_settings)

In [5]:
try:  # Needed to handle out of order cell run in Jupyter
    sim.close()
except NameError:
    pass
sim = habitat_sim.Simulator(cfg)


Renderer: NVIDIA GeForce RTX 3060 Laptop GPU/PCIe/SSE2 by NVIDIA Corporation
OpenGL version: 4.6.0 NVIDIA 560.35.03
Using optional features:
    GL_ARB_vertex_array_object
    GL_ARB_separate_shader_objects
    GL_ARB_robustness
    GL_ARB_texture_storage
    GL_ARB_texture_view
    GL_ARB_framebuffer_no_attachments
    GL_ARB_invalidate_subdata
    GL_ARB_texture_storage_multisample
    GL_ARB_multi_bind
    GL_ARB_direct_state_access
    GL_ARB_get_texture_sub_image
    GL_ARB_texture_filter_anisotropic
    GL_KHR_debug
    GL_KHR_parallel_shader_compile
    GL_NV_depth_buffer_float
Using driver workarounds:
    no-forward-compatible-core-context
    nv-egl-incorrect-gl11-function-pointers
    no-layout-qualifiers-on-old-glsl
    nv-zero-context-profile-mask
    nv-implementation-color-read-format-dsa-broken
    nv-cubemap-inconsistent-compressed-image-size
    nv-cubemap-broken-full-compressed-image-query
    nv-compressed-block-size-in-bits


PluginManager::Manager: duplicate static plugin StbImageImporter, ignoring
PluginManager::Manager: duplicate static plugin GltfImporter, ignoring
PluginManager::Manager: duplicate static plugin BasisImporter, ignoring
PluginManager::Manager: duplicate static plugin AssimpImporter, ignoring
PluginManager::Manager: duplicate static plugin AnySceneImporter, ignoring
PluginManager::Manager: duplicate static plugin AnyImageImporter, ignoring
[13:14:21:444494]:[Warning]:[Metadata] SceneDatasetAttributes.cpp(107)::addNewSceneInstanceToDataset : Dataset : 'default' : Lighting Layout Attributes 'no_lights' specified in Scene Attributes but does not exist in dataset, so creating default.


In [10]:
import habitat_sim
import numpy as np
import matplotlib.pyplot as plt
import cv2  # For video processing
import random  # For generating random actions

# Assuming sim and sim_settings are already defined, with a valid config

# Get the discrete actions for the agent (move_forward, turn_left, turn_right)
action_names = list(cfg.agents[sim_settings["default_agent"]].action_space.keys())
print("Discrete action space: ", action_names)

# Number of agents
num_agents = 2

# Initialize the agents' states with different positions
starting_positions = [
    np.array([0.0,0.0,0.0]),  # Starting position for agent 0
    np.array([1.0, 1.0, 0.0])   # Starting position for agent 1
]

# Initialize and store states for both agents and their paths
agents_states = []
agents_paths = [[] for _ in range(num_agents)]  # List of paths for each agent
for i in range(num_agents):
    agent = sim.initialize_agent(sim_settings["default_agent"])
    agent_state = habitat_sim.AgentState()
    agent_state.position = starting_positions[i]
    agent.set_state(agent_state)  # Set the initial state of the agent
    agents_states.append(agent_state)  # Store the agent's state

# Define a large number of random actions for each agent
num_steps = 100  # Set the number of steps for each agent
actions_per_agent = [
    [random.choice(action_names) for _ in range(num_steps)] for _ in range(num_agents)
]

# Video writer setup
frame_width, frame_height = 640, 480  # Adjust according to your observation size
combined_frame_width = frame_width * 3  # Two agent views + top-down map
out = cv2.VideoWriter('combined_agent_paths.avi', cv2.VideoWriter_fourcc(*'XVID'), 20.0, (combined_frame_width, frame_height))

# convert 3d points to 2d topdown coordinates
def convert_points_to_topdown(pathfinder, points, meters_per_pixel):
    points_topdown = []
    bounds = pathfinder.get_bounds()
    for point in points:
        # convert 3D x,z to topdown x,y
        px = (point[0] - bounds[0][0]) / meters_per_pixel
        py = (point[2] - bounds[0][2]) / meters_per_pixel
        points_topdown.append(np.array([px, py]))
    return points_topdown
# Updated top-down map generation function
def generate_topdown_map(agents_paths, height, meters_per_pixel=0.1):
    # Convert points to top-down map visualization
    top_down_map = maps.get_topdown_map(sim.pathfinder, height=height, meters_per_pixel=meters_per_pixel)
    recolor_map = np.array([[255, 255, 255], [128, 128, 128], [0, 0, 0]], dtype=np.uint8)
    top_down_map = recolor_map[top_down_map]

    # Overlay each agent's path and current position on the top-down map
    colors = [(255, 0, 0), (0, 0, 255)]  # Red for agent 0, blue for agent 1
    for i, agent_path in enumerate(agents_paths):
        if len(agent_path) > 1:
            # Draw the path as a line connecting all positions
            for j in range(1, len(agent_path)):
                start_pos = tuple(map(int, convert_points_to_topdown(sim.pathfinder, [agent_path[j - 1]], meters_per_pixel)[0]))
                end_pos = tuple(map(int, convert_points_to_topdown(sim.pathfinder, [agent_path[j]], meters_per_pixel)[0]))
                cv2.line(top_down_map, start_pos, end_pos, colors[i], thickness=2)

        # Draw the current position as a dot
        current_pos = tuple(map(int, convert_points_to_topdown(sim.pathfinder, [agent_path[-1]], meters_per_pixel)[0]))
        cv2.circle(top_down_map, current_pos, 5, colors[i], -1)  # -1 fills the circle

    return top_down_map

# Function to execute actions for each agent and capture frames
def navigate_and_see(agent_index, action):
    if action in action_names:
        # Restore the agent's state before performing the action
        agent_state = agents_states[agent_index]
        agent = sim.initialize_agent(sim_settings["default_agent"])
        agent.set_state(agent_state)
        
        # Perform the action and get the observations
        observations = sim.step(action)
        
        # Store the updated agent state after the action
        agent_state = agent.get_state()
        agents_states[agent_index] = agent_state
        
        # Store the current position in the agent's path
        agents_paths[agent_index].append(agent_state.position.copy())
        
        # Capture RGB observation
        rgb_image = observations["color_sensor"]  # Get the RGB image from observations
        rgb_image_bgr = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)  # Convert from RGB to BGR for OpenCV
        return rgb_image_bgr

# Main loop to execute actions for both agents and build video frames
for step in range(num_steps):
    agent_frames = []
    for i in range(num_agents):
        action = actions_per_agent[i][step]
        agent_frame = navigate_and_see(i, action)
        agent_frames.append(agent_frame)

    # Generate the top-down map with updated agent paths
    top_down_map_frame = generate_topdown_map(agents_paths, starting_positions[0][1])

    # Ensure the top-down map has the same height as agent frames
    top_down_map_frame = cv2.resize(top_down_map_frame, (frame_width, frame_height))

    # Resize agent frames if necessary to match height
    for idx in range(len(agent_frames)):
        if agent_frames[idx].shape[:2] != (frame_height, frame_width):
            agent_frames[idx] = cv2.resize(agent_frames[idx], (frame_width, frame_height))

    # Combine all frames horizontally
    combined_frame = np.hstack((agent_frames[0], agent_frames[1], top_down_map_frame))

    # Write the combined frame to the video
    out.write(combined_frame)

# Release video writer
out.release()
print("Combined video saved as 'combined_agent_paths.avi'.")


Discrete action space:  ['move_forward', 'turn_left', 'turn_right']
Combined video saved as 'combined_agent_paths.avi'.
